In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def hash_mult_add(item, m):
    hash_value = 0
    for char in str(item):
        hash_value = (hash_value * 31 + ord(char)) 
    return hash_value % m

def hash_xor(item, m):
    hash_value = 0
    for char in str(item):
        hash_value ^= (hash_value << 5) + ord(char) + (hash_value >> 2)
    return hash_value % m

class HyperLogLog:
    def __init__(self, b=4, hash_fn=hash_mult_add):
        self.b = b
        self.m = 2 ** b  
        self.registers = [0] * self.m
        self.alpha = self._get_alpha()
        self.hash_fn = hash_fn

    def _get_alpha(self):
        if self.m == 16:
            return 0.673
        elif self.m == 32:
            return 0.697
        elif self.m == 64:
            return 0.709
        else:
            return 0.7213 / (1 + 1.079 / self.m)

    def _hash(self, item):
        return self.hash_fn(item, 1 << 64) 

    def add(self, item):
        hash_value = self._hash(item)
        index = hash_value & (self.m - 1) 
        w = hash_value >> self.b 
        self.registers[index] = max(self.registers[index], self._count_leading_zeros(w) + 1)

    def _count_leading_zeros(self, w):
        if w == 0:
            return 64 - self.b
        return bin(w)[2:].zfill(64 - self.b).index('1')

    def count(self):
        harmonic_mean = sum(2 ** -r for r in self.registers)
        estimate = self.alpha * self.m ** 2 / harmonic_mean

        if estimate <= 2.5 * self.m:
            zeros = self.registers.count(0)
            if zeros != 0:
                return self.m * np.log(self.m / zeros)
        elif estimate > (1 << 64) / 30:
            return -(1 << 64) * np.log(1 - estimate / (1 << 64))
        return estimate

def evaluate_hyperloglog_accuracy(n, b_values, hash_fn):
    results = []
    for b in b_values:
        hll = HyperLogLog(b=b, hash_fn=hash_fn)
        elements = np.random.randint(0, 1000000, n)
        unique_elements = set(elements)  
        for element in elements:
            hll.add(element)
        estimated_count = hll.count()
        true_count = len(unique_elements)
        error = abs(estimated_count - true_count) / true_count
        results.append((b, estimated_count, true_count, error))
    return results

n = 100000  
b_values = [4, 6, 8, 10] 

hash_functions = {
    "hash_mult_add": hash_mult_add,
    "hash_xor": hash_xor,
}

for hash_name, hash_fn in hash_functions.items():
    print(f"Результаты для хеш-функции: {hash_name}")
    results = evaluate_hyperloglog_accuracy(n, b_values, hash_fn)

    df = pd.DataFrame(results, columns=["b", "Оценка", "Истинное значение", "Ошибка"])
    print(df)

    plt.figure(figsize=(10, 6))

    m_list = [2 ** b for b in b_values]
    error_list = [error for (b, est, true, error) in results]
    
    plt.plot(m_list, error_list, marker='o', label=f"{hash_name}")

    plt.xlabel("Размер массива (m = 2^b)")
    plt.ylabel("Ошибка оценки")
    plt.title(f"Зависимость ошибки оценки от размера массива (m) ({hash_name})")
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
hll = HyperLogLog(b=4, hash_fn=hash_mult_add)

words = ["apple", "banana", "cherry", "date", "elderberry"]
for word in words:
    hll.add(word)

estimated_count = hll.count()
print(f"Оценка количества уникальных элементов: {estimated_count}")

True
False
False
